Hello world from (Scala) Spark!

In [ ]:
println("Hello, world!")

A simple word count

In [ ]:
val sentence: String = "tre tigri contro tre tigri"
val res: Array[(String, Int)] = sc.parallelize(sentence.split(" ")).map((_, 1)).reduceByKey(_ + _).collect()

Save the result to HDFS

In [ ]:
import org.apache.hadoop.fs.{FileSystem, Path}
val fs = FileSystem.get(sc.hadoopConfiguration) // get the file system
val hdfspath: String = "wordcount" // HDFS path

val outPutPath = new Path(hdfspath)
if (fs.exists(outPutPath)) { // delete the HDFS folder if exists
    fs.delete(outPutPath, true)
}

def writeandread(path: String) = {
    sc.parallelize(res).saveAsTextFile(path) // save the RDD
    val rdd = sc.textFile(path) // read it back
    rdd.collect() // print it 
}

writeandread(hdfspath)

... and to S3 as well

In [ ]:
val s3bucket: String = "aws-emr-resources-604905954159-us-east-1"
val s3path: String = s"s3://${s3bucket}/wordcount"
writeandread(s3path)

From Spark to Hive

In [ ]:
import org.apache.spark.sql.hive.HiveContext
val hiveContext = new HiveContext(sc)
import hiveContext.implicits._

sc.textFile(s"s3://${s3bucket}/inferno.txt").flatMap(_.split(" ")).filter(_.nonEmpty).map((_, 1)).reduceByKey(_ + _).toDF("word", "count").write.mode("overwrite").saveAsTable("inferno")

In [ ]:
sql("select word, count from inferno order by 2 desc").show()